## Problem Set 9
### Ryan Wills

In [29]:
import numpy as np
import random

In [30]:
x = ['STEP']
y = ['APE']

In [31]:
def optimalOp(S, i, j, x, y):
    
    #initialize costs from the homework
    swapCost = 5
    subCost = 3
    insertCost = 1
    deleteCost = 1
    
    #initialize lists to hold all the possible alignment/edit operations
    opList = []
    valueList = []
    

    insertValue = S[i][j - 1]
    valueList.append(insertValue)
    #coming from the top
    #deleteValue = S[i][j - 1]
    deleteValue = S[i - 1][j]
    valueList.append(deleteValue)
    #coming from the diagonal
    subValue = S[i - 1][j - 1]
    valueList.append(subValue)
            
    if (subValue + subCost == S[i][j]):
        opList.append("sub")
    
    if (insertValue + insertCost == S[i][j]):
        opList.append("insert")
    
    if (deleteValue + deleteCost == S[i][j]):
        opList.append("delete")

    #check if a swap was possible
    if (i >= 2 and j >= 2):
        
        #check the possible sub costs
        possibleSubs = list(set(x[j - 2:j]) & set(y[i - 2:i]))
        
        subCost = (2 - len(possibleSubs)) * 3
        swap = S[i-2][j-2]
        if (swap + swapCost+ subCost == S[i][j]):
            print("adding a swap")
            opList.append("swap")
            
        
    if len(opList) == 0:
        return "no-op"
    
    #randomly choose an element from the list if more than 1 is in there
    return random.choice(opList)


In [32]:
def alignStrings(x,y):
    nX = len(x)
    nY = len(y)
    
    S = [[nX+1],[nY+1]] # for memoizing the subproblem costs
    S[0] = list(np.arange(nX))
    S[1] = list(np.arange(nY))            # fill in the base cases
    
    for i in range (1, nX):
        row = S[i]
        
        for j in range (1, nY):
            
            if (x[i - 1] == y[i - 1]):
                
                diagonal = S[j - 1][i  - 1]
                row[j] = diagonal
                
            #characters don't match so find the min from the top/left/diagonal
            else:
                
                diagonalValue = S[i - 1][j  - 1] + 3
               
                topValue = S[i - 1][j] + 1
               
                leftValue = row[j - 1] + 1
                
                #check to see if you can calculate swap value
                if i >= 2 and j >= 2:
                    #use python sets to see which characters you have in common:
                    commonChar = list(set(x[j - 2:j]) & set(y[i - 2:i]))
                    
                    subCost = (2 - len(commonChar)) * 3
                   
                    swapValue = S[i - 2][j - 2] + 5 + subCost
                    minCost = min(topValue, leftValue, diagonalValue, swapValue)
                    
                    row[columnIndex] = minCost
                    
                
                else:
                    minCost = min(topValue, leftValue, diagonalValue)
                   
                    row[i] = minCost       
        
    return S

In [33]:
def updateIndices(i, j, a):
    #the last alignment operation added to the list is the one I want to use to move to the next cell
    operation = a[-1]
    
    if operation == "insert":
        j -= 1
        return (i),(j)
    
    elif operation == "delete":
        i -= 1
        return (i),(j)
    
    elif operation == "no-op" or operation == "sub":
        i -= 1
        j -= 1
        return (i),(j)
    
    else:
        i -= 2
        j -= 2
        return (i),(j)

In [34]:
def extractAlignment(S, x, y):
    #empty list of operations
    editOpList = []
    
    #set position to lower right corner
    i = len(y)
    j = len(x)
    startPosition = S[i][j]
    
    while (i > 0 or j > 0):
        editOpList.append(optimalOp(S, i, j, x, y))
        i,j = updateIndices(i, j, editOpList)
        currentPosition = S[i][j]
        
    return editOpList

In [35]:
def commonSubstrings(stringToTransform, L, editList):
    
    #have to reverse my list of edit operations
    a.reverse()
    #store the substrings in the list
    substrings = []
    #string variable to concatenate all possible no-op substrings
    tempSubstring = ""
        
    #loop through all the edit operations
    for eachLetter in range(len(a)):
        #delete in one string corresponds to an insert in the other
        if (a[eachLetter] == 'delete'):
            #need to add a gap into my string or else I'll index out of range
            x = x[:eachLetter] + "_" + x[eachLetter:]
            
            #check if the substring is large enough to be considered
            if len(tempSubstring) >= L:
                substrings.append(tempSubstring)
            
            #clearing the substring variable
            tempSubstring = ''
            
        #if no-op then I pulled from the diagonal
        if (a[eachLetter] == 'no-op'):
            #concatentate my string
            tempSubstring += x[eachLetter]               
    
        #if not a no-op then I need to end my substring --> make sure to clear it from previous values
        else:
            #check if the substring is large enough to be considered
            if len(tempSubstring) >= L:
                substrings.append(tempSubstring)
            #clearing the substring variable
            tempSubstring = ''
            
    #need to check one last time in case it ends on a no-op
    if (len(tempSubstring) >= L):
        substrings.append(tempSubstring)
            
    return(substrings)